In [33]:
# %pip install bm25s
# %pip install spacy
# %pip install -U 'spacy[cuda12x]'
# %pip install rouge_score
# %pip install pysbd

In [1]:
import functions as fct
import bm25s
import numpy as np
import json
import time
import os

resource module not available on Windows


# BM25

In [35]:
# # Définition des chemins vers les fichiers JSON d'entraînement et de développement
# train_path = 'SCOTUS/train.json'
# dev_path = 'SCOTUS/dev.json'

# # Ouverture du fichier d'entraînement
# train = fct.open_file(train_path, "json")

# # Récupération du document source du premier élément de l'ensemble d'entraînement
# # document = train[0]["raw_source"]
# document = train

# # Récupération des éléments de la cible (faits, question, conclusion)
# paragraph_target = (
#     train[0]['raw_target']['facts_of_the_case'] +
#     train[0]['raw_target']['question'] +
#     train[0]['raw_target']['conclusion']
# )

# train_path = 'data_txt_save/train_0.txt'
# train = fct.open_file(train_path, "txt")

# document = train


# # Segmentation des phrases du document source
# sentences = fct.sent_segmentation(document, method='custom_spacy')

# # Résumé des phrases en utilisant le modèle BERT
# summary = fct.bb25LegalSum(sentences, "bert-base-uncased", 5)

# # Évaluation de la qualité du résumé à l'aide de la métrique ROUGE
# bb25_rouge = fct.rouge_evaluations(" ".join(summary), paragraph_target)

In [36]:
# bb25_rouge

,Metric,Precision,Recall,F1-Score
0,rouge1,0.394928,0.605556,0.478070
1,rouge2,0.138182,0.212291,0.167401
2,rougeL,0.170290,0.261111,0.206140


In [2]:
# Définition des chemins vers les fichiers JSON d'entraînement et de développement
train_path_json = 'SCOTUS/train.json'
dev_path_json = 'SCOTUS/dev.json'

# Ouverture du fichier d'entraînement JSON
train_json = fct.open_file(train_path_json, "json")

# Récupération du document source et des éléments de la cible (faits, question, conclusion)
document_json = train_json[0]["raw_source"]
paragraph_target_json = (
    train_json[0]['raw_target']['facts_of_the_case'] +
    train_json[0]['raw_target']['question'] +
    train_json[0]['raw_target']['conclusion']
)

# Définir le chemin du fichier texte à traiter
train_path_txt = 'data_txt_save/train_0.txt'
clean_path_txt = 'clean_data_txt_save/train_0.txt'

# Ouverture du fichier d'entraînement TXT
document_txt = fct.open_file(train_path_txt, "txt")
clean_document_txt = fct.open_file(clean_path_txt, "txt")

# Liste des méthodes de segmentation et des modèles à tester
methods = ['nltk', 'spacy', 'pySBD', 'custom_spacy']  # Méthodes de segmentation à tester
model = "bert-base-uncased"  # Modèle de résumé à utiliser
results = {}  # Dictionnaire pour stocker les résultats
execution_times = {}  # Dictionnaire pour stocker les temps d'exécution

def save_txt(text, folder_name, file_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    file_path = os.path.join(folder_name, file_name)

    with open(file_path, 'w') as file:
        file.write(text)

# Fonction pour évaluer les modèles
def evaluate_models(document, paragraph_target, file_type):
    for method in methods:
        start_time = time.time()

        # Segmentation des phrases du document source
        sentences = fct.sent_segmentation(document, method=method)
        
        # Résumé des phrases
        summary = fct.bb25LegalSum(sentences, model, 5)
        
        # Évaluation de la qualité du résumé à l'aide de la métrique ROUGE
        bb25_rouge = fct.rouge_evaluations(" ".join(summary), paragraph_target)

        end_time = time.time()
        execution_time = end_time - start_time
        
        # summary_conc = "\n".join(summary)
        # save_txt(summary_conc, f"summaries/{method}/", "summary_train_0.txt") #save train_0 summary
        
        # Stockage des résultats dans le dictionnaire
        results[f"{file_type} - {method}"] = bb25_rouge
        execution_times[f"{file_type} - {method}"] = execution_time

# Évaluation des modèles pour le fichier JSON
evaluate_models(document_json, paragraph_target_json, "JSON")

# Évaluation des modèles pour le fichier TXT
evaluate_models(document_txt, paragraph_target_json, "TXT")  # Utilise le même paragraphe cible
evaluate_models(clean_document_txt, paragraph_target_json, "Cleaned TXT")  # Utilise le même paragraphe cible

In [72]:
import pandas as pd
dfs = []

for key, value in results.items():
    df = pd.DataFrame(value)  # Convert the dictionary to a DataFrame
    df['Method'] = key        # Add a 'Method' column
    df['Execution Time'] = execution_times.get(key, None)  # Add the execution time
    dfs.append(df) 

# Combine the DataFrames
final_df = pd.concat(dfs, ignore_index=True)

# Rearrange the columns for better readability
final_df = final_df[['Method', 'Metric', 'Precision', 'Recall', 'F1-Score', 'Execution Time']]

def highlight_min_max(df):
    styles = pd.DataFrame('', index=df.index, columns=df.columns)

    # Appliquer le style pour les colonnes 'Precision', 'Recall', 'F1-Score'
    for col in ['Precision', 'Recall', 'F1-Score']:
        # Top 3 maximums et minimums
        top_3_max = df[col].nlargest(3)
        top_3_min = df[col].nsmallest(3)

        # Appliquer le dégradé vert pour les max
        for i in df.index:
            if df[col].iloc[i] in top_3_max.values:
                rank = top_3_max.rank(ascending=False)[top_3_max == df[col].iloc[i]].values[0]
                alpha = 1 - (rank - 1) / 3
                styles.loc[i, col]= f'background-color: rgba(50, 200, 50, {alpha});'

        # Appliquer le dégradé rouge pour les min
        for i in df.index:
            if df[col].iloc[i] in top_3_min.values:
                rank = top_3_min.rank()[top_3_min == df[col].iloc[i]].values[0]
                alpha = 1 - (rank - 1) / 3  
                styles.loc[i, col]= f'background-color: rgba(200, 50, 50, {alpha});'

    # Pour la colonne 'Execution Time', inverser les couleurs (max en rouge, min en vert)
    col = 'Execution Time'
    top_3_max = df[col].nlargest(3)
    top_3_min = df[col].nsmallest(3)

    # Appliquer le dégradé rouge pour les max
    for i in df.index:
        if df[col].iloc[i] in top_3_max.values:
            rank = top_3_max.rank(ascending=False)[top_3_max == df[col].iloc[i]].values[0]
            alpha = 1 - (rank - 1) / 3 
            styles.loc[i, col]= f'background-color: rgba(200, 50, 50, {alpha});'

    # Appliquer le dégradé vert pour les min
    for i in df.index:
        if df[col].iloc[i] in top_3_min.values:
            rank = top_3_min.rank()[top_3_min == df[col].iloc[i]].values[0]
            alpha = 1 - (rank - 1) / 3 
            styles.loc[i, col]= f'background-color: rgba(50, 200, 50, {alpha});'
                
    # Ajouter des lignes grises entre les méthodes
    for i in range(1, len(df)):
        if df['Method'].iloc[i] != df['Method'].iloc[i - 1]:
            styles.iloc[i - 1] += 'border-bottom: 2px solid gray'

    return styles

styled_df = final_df.style.apply(highlight_min_max, axis=None)

styled_df

,Method,Metric,Precision,Recall,F1-Score,Execution Time
0,JSON - nltk,rouge1,0.550725,0.046441,0.085658,13.142880
1,JSON - nltk,rouge2,0.149091,0.012531,0.023118,13.142880
2,JSON - nltk,rougeL,0.210145,0.017721,0.032685,13.142880
3,JSON - spacy,rouge1,0.072464,0.312500,0.117647,56.986502
4,JSON - spacy,rouge2,0.010909,0.047619,0.017751,56.986502
5,JSON - spacy,rougeL,0.050725,0.218750,0.082353,56.986502
6,JSON - pySBD,rouge1,0.090580,0.324675,0.141643,57.636624
7,JSON - pySBD,rouge2,0.018182,0.065789,0.028490,57.636624
8,JSON - pySBD,rougeL,0.057971,0.207792,0.090652,57.636624
9,JSON - custom_spacy,rouge1,0.112319,0.430556,0.178161,60.185831
